<a href="https://colab.research.google.com/github/udlbook/udlbook/blob/main/Notebooks/Chap12/12_3_Tokenization.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Notebook 12.3: Tokenization**

This notebook builds set of tokens from a text string as in figure 12.8 of the book.

Work through the cells below, running each cell in turn. In various places you will see the words "TO DO". Follow the instructions at these places and make predictions about what is going to happen or write code to complete the functions.

I adapted this code from *SOMEWHERE*.  If anyone recognizes it, can you let me know and I will give the proper attribution or rewrite if the license is not permissive.

Contact me at udlbookmail@gmail.com if you find any mistakes or have any suggestions.



In [1]:
import re, collections
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [2]:
text = "a sailor went to sea sea sea "+\
                  "to see what he could see see see "+\
                  "but all that he could see see see "+\
                  "was the bottom of the deep blue sea sea sea"

Tokenize the input sentence To begin with the tokens are the individual letters and the </w> whitespace token. So, we represent each word in terms of these tokens with spaces between the tokens to delineate them.

The tokenized text is stored in a structure that represents each word as tokens together with the count of how often that word occurs.  We'll call this the *vocabulary*.

In [3]:
def initialize_vocabulary(text):
  vocab = collections.defaultdict(int)
  words = text.strip().split()
  for word in words:
      # note that ' '.join() means pad with white space between the characters, list here is the same as splitting up the words into characters try for instance list("hello") to see what I mean
      vocab[' '.join(list(word)) + ' </w>'] += 1
  return vocab

In [4]:
vocab = initialize_vocabulary(text)
print('Vocabulary: {}'.format(vocab))
print('Size of vocabulary: {}'.format(len(vocab)))

Vocabulary: defaultdict(<class 'int'>, {'a </w>': 1, 's a i l o r </w>': 1, 'w e n t </w>': 1, 't o </w>': 2, 's e a </w>': 6, 's e e </w>': 7, 'w h a t </w>': 1, 'h e </w>': 2, 'c o u l d </w>': 2, 'b u t </w>': 1, 'a l l </w>': 1, 't h a t </w>': 1, 'w a s </w>': 1, 't h e </w>': 2, 'b o t t o m </w>': 1, 'o f </w>': 1, 'd e e p </w>': 1, 'b l u e </w>': 1})
Size of vocabulary: 18


### Why create tokens like this i.e. padding each letter with space?
<span style="color:green;white-space:pre-wrap">One might ask why we create the tokens this way, by adding white spaces between each character. It seems that later on they want to know the frequencies of each character and also what other character follows it, so it makes it easier to look into that when the tokens are constructed this way.</span>

## Find all single letter frequencies
Find all the tokens in the current vocabulary and their frequencies.
<span style="color:green;white-space:pre-wrap">Interpretation, the less white spaces there are in the keys of the vocab, the fewer number of tokens. This happens when merging tokens together, always looking for adjacent pairs. So as the iteration increases in the merging process, the fewer tokens there will be that have sub-words or letters in them that are split by whitespace.</span>

In [5]:
def get_tokens_and_frequencies(vocab):
  tokens = collections.defaultdict(int)
  for word, freq in vocab.items():
      word_tokens = word.split()
      for token in word_tokens:
          tokens[token] += freq
  return tokens

In [6]:
tokens = get_tokens_and_frequencies(vocab)
print('Tokens: {}'.format(tokens))
print('Number of tokens: {}'.format(len(tokens)))

Tokens: defaultdict(<class 'int'>, {'a': 12, '</w>': 33, 's': 15, 'i': 1, 'l': 6, 'o': 8, 'r': 1, 'w': 3, 'e': 28, 'n': 1, 't': 11, 'h': 6, 'c': 2, 'u': 4, 'd': 3, 'b': 3, 'm': 1, 'f': 1, 'p': 1})
Number of tokens: 19


## Find adjacent pair (two letters adjacent to each other) frequency
Find each pair of adjacent tokens in the vocabulary
and count them.  We will subsequently merge the most frequently occurring pair.

<span style="color:green;white-space:pre-wrap">It's not apparent, but what the split does it to split at white characters. But that means tokens such as 's e e' and 's e a' will add more to the frequency of ('s', e'), meaning that ('s', 'e') will have contributions of more than just one token. This explains why using the initial vocab to create pairs we have as much as 13 ('s', 'e') pairs. Note that there's an edge case here, if there are no white spaces within the keys of the vocab, i.e. we have converged by merging everything, then the word.split() will return just the word. But the loop will see that the len(word) is just 1 because you got one word in a list and so subtracting that with 1 gives zero. This will happen to every key now that everything has been merged, so we will only return an empty pair dictionary. Later on this will cause issue when we try to take max frequency of this pair dictionary. That's what is causing the error.</span>

In [7]:
def get_pairs_and_counts(vocab):
    pairs = collections.defaultdict(int)
    for word, freq in vocab.items():
        symbols = word.split()
        # so this splits each key of the vocab by the white space, meaning that keys that have overlapping sub-words will contribute much more to those pairs
        # in this case 's e e' and 's e a' will contribute to the frequency of the pair ('s', 'e')
        # anyway, any pair will be constructed given a key as long as there are white spaces between the characters in the key
        for i in range(len(symbols)-1):
            pairs[symbols[i],symbols[i+1]] += freq
    return pairs

In [8]:
pairs = get_pairs_and_counts(vocab)
print('Pairs: {}'.format(pairs))
print('Number of distinct pairs: {}'.format(len(pairs)))

most_frequent_pair = max(pairs, key=pairs.get)
print('Most frequent pair: {}'.format(most_frequent_pair))

Pairs: defaultdict(<class 'int'>, {('a', '</w>'): 7, ('s', 'a'): 1, ('a', 'i'): 1, ('i', 'l'): 1, ('l', 'o'): 1, ('o', 'r'): 1, ('r', '</w>'): 1, ('w', 'e'): 1, ('e', 'n'): 1, ('n', 't'): 1, ('t', '</w>'): 4, ('t', 'o'): 3, ('o', '</w>'): 2, ('s', 'e'): 13, ('e', 'a'): 6, ('e', 'e'): 8, ('e', '</w>'): 12, ('w', 'h'): 1, ('h', 'a'): 2, ('a', 't'): 2, ('h', 'e'): 4, ('c', 'o'): 2, ('o', 'u'): 2, ('u', 'l'): 2, ('l', 'd'): 2, ('d', '</w>'): 2, ('b', 'u'): 1, ('u', 't'): 1, ('a', 'l'): 1, ('l', 'l'): 1, ('l', '</w>'): 1, ('t', 'h'): 3, ('w', 'a'): 1, ('a', 's'): 1, ('s', '</w>'): 1, ('b', 'o'): 1, ('o', 't'): 1, ('t', 't'): 1, ('o', 'm'): 1, ('m', '</w>'): 1, ('o', 'f'): 1, ('f', '</w>'): 1, ('d', 'e'): 1, ('e', 'p'): 1, ('p', '</w>'): 1, ('b', 'l'): 1, ('l', 'u'): 1, ('u', 'e'): 1})
Number of distinct pairs: 48
Most frequent pair: ('s', 'e')


## Create frequency dict of tokens and only merge most common token
Merge the instances of the best pair in the vocabulary

<span style="color:green;white-space:pre-wrap">So at first I thought this was worthless and nothing changed, but I doubled checked and saw that actually any word token that has 's' and 'e' as adjacent pair (because we input max frequency pair which happened to be ('s','e'), where they are surrounded by whitespace will actually get merged. So that's the only change happening. The rest of the tokens are the same. What the output of this is the frequencies of the tokens in the vocabulary including the frequency of the new merged token that has 's' 'e' in it. Also, the reason we condition this on the most frequent pair everytime is because merged words are expected to be fewer than the unmerged words, because the vocab started with keys in which each character were padded by whitespaces, so none of them started as words at the initialization of the keys of the vocab (they obviously started as words in the text that the vocab used as input).</span>

In [9]:
def merge_pair_in_vocabulary(pair, vocab_in):
    vocab_out = {}
    bigram = re.escape(' '.join(pair))  # escaping the most frequent pair that is ('s', 'e') and padding white space between them, creating 's \ e'
    # (?<!B)A means negative lookbehind, can think of it as < pointing to behind. It means find expression A whatever where B does not precede 
    # A(?!B) means negative lookahead, means find expression A where B does not follow
    # In this case it means that it matches the bigram that is NOT preceded by a non-whitespace character and NOT followed by a non-whitespace character,
    # so essentially we are looking for the bigram that is SURROUNDED by space characters
    p = re.compile(r'(?<!\S)' + bigram + r'(?!\S)')
    for word in vocab_in:
        # in sub re.sub(pattern, replacement, text), we already specified the pattern above, 
        # so now we want to look for all instances in the word that matches with the pattern "bigram surrounded by white space", 
        # and replace those instances with the actual bigram. The bigram in this case is the most frequent pair, which is 'se' after the join ''.() transformation
        # in essence what we are doing is removing the white space surrounding the bigram, 
        # and we are doing this for all occurences of 's e' and replace them with 'se', i.e. we are merging the bigram
        # print(f"before {word}")
        word_out = p.sub(''.join(pair), word)
        # print(f"after {word_out}")
        # then we associate frequency of the word with the bigram, where bigram is the key and frequency the value
        vocab_out[word_out] = vocab_in[word]
    return vocab_out

In [10]:
# This is important to mention, but all variables in jupyter notebook are global variables, so mutating vocab will change it forever, even if I rerun this cell. It will be changed for the duration of the instance.
# One way to solve this is to just rerun the entire notebook, especially the part where we reinitialize the vocab.
print(vocab)
vocab = merge_pair_in_vocabulary(most_frequent_pair, vocab)
print('Vocabulary: {}'.format(vocab))
print('Size of vocabulary: {}'.format(len(vocab)))

defaultdict(<class 'int'>, {'a </w>': 1, 's a i l o r </w>': 1, 'w e n t </w>': 1, 't o </w>': 2, 's e a </w>': 6, 's e e </w>': 7, 'w h a t </w>': 1, 'h e </w>': 2, 'c o u l d </w>': 2, 'b u t </w>': 1, 'a l l </w>': 1, 't h a t </w>': 1, 'w a s </w>': 1, 't h e </w>': 2, 'b o t t o m </w>': 1, 'o f </w>': 1, 'd e e p </w>': 1, 'b l u e </w>': 1})
Vocabulary: {'a </w>': 1, 's a i l o r </w>': 1, 'w e n t </w>': 1, 't o </w>': 2, 'se a </w>': 6, 'se e </w>': 7, 'w h a t </w>': 1, 'h e </w>': 2, 'c o u l d </w>': 2, 'b u t </w>': 1, 'a l l </w>': 1, 't h a t </w>': 1, 'w a s </w>': 1, 't h e </w>': 2, 'b o t t o m </w>': 1, 'o f </w>': 1, 'd e e p </w>': 1, 'b l u e </w>': 1}
Size of vocabulary: 18


Update the tokens, which now include the best token 'se'

In [11]:
tokens = get_tokens_and_frequencies(vocab)
print('Tokens: {}'.format(tokens))
print('Number of tokens: {}'.format(len(tokens)))

Tokens: defaultdict(<class 'int'>, {'a': 12, '</w>': 33, 's': 2, 'i': 1, 'l': 6, 'o': 8, 'r': 1, 'w': 3, 'e': 15, 'n': 1, 't': 11, 'se': 13, 'h': 6, 'c': 2, 'u': 4, 'd': 3, 'b': 3, 'm': 1, 'f': 1, 'p': 1})
Number of tokens: 20


Now let's write the full tokenization routine
<span style="color:green;white-space:pre-wrap">Note that when the vocab and tokens have converged, but you still want to continue to loop an error will occur on max simply because the vocab will be empty, so get_pairs method will not be able to return a pair dictionary, it will just be empty. This means that when max tries to get something out of the empty dictionary an error will be raised. The reason that we get an empty dictionary is when we have converged there will be no whitespace within the keys of the vocab and thus consequently the pair dictionary. When trying to split each word that is a key that has no white space, the entire word will be returned. In the loop that creates the pairs the length of this list of split characters/words will just be 1, but we are always subtracting that length by 1, which gives zero. So the loop will never run and create pairs. Therefore, an empty dictionary of pairs will be returned. Thus, when we try to take max frequency of this dictionary an error will occur. </span>

In [12]:
# TODO -- write this routine by filling in this missing parts,
# calling the above routines
def tokenize(text, num_merges):
  # Initialize the vocabulary from the input text
  vocab = initialize_vocabulary(text)
  # Find the tokens initially and how often they occur in the vocabulary
  tokens = get_tokens_and_frequencies(vocab)
  
  # print initial stats of size of vocab and number of tokens
  print("Initial stats of size of vocab and number of tokens")
  print('Tokens: {}'.format(tokens))
  print('Number of tokens: {}'.format(len(tokens)))
  print('Vocabulary: {}'.format(vocab))
  print('Size of vocabulary: {}'.format(len(vocab)))
  
  for i in range(num_merges):
    # Find the pairs of adjacent tokens and their counts
    pairs = get_pairs_and_counts(vocab)

    # Find the most frequent pair
    most_frequent_pair = max(pairs, key=pairs.get)
    print('Most frequent pair: {}'.format(most_frequent_pair))

    # Merge the code in the vocabulary
    vocab = merge_pair_in_vocabulary(most_frequent_pair, vocab)

  # Find the tokens and how often they occur in the vocabulary one last time
  tokens = get_tokens_and_frequencies(vocab)

  return tokens, vocab

In [13]:
tokens, vocab = tokenize(text, num_merges=50)
print('Tokens: {}'.format(tokens))
print('Number of tokens: {}'.format(len(tokens)))
print('Vocabulary: {}'.format(vocab))
print('Size of vocabulary: {}'.format(len(vocab)))

Initial stats of size of vocab and number of tokens
Tokens: defaultdict(<class 'int'>, {'a': 12, '</w>': 33, 's': 15, 'i': 1, 'l': 6, 'o': 8, 'r': 1, 'w': 3, 'e': 28, 'n': 1, 't': 11, 'h': 6, 'c': 2, 'u': 4, 'd': 3, 'b': 3, 'm': 1, 'f': 1, 'p': 1})
Number of tokens: 19
Vocabulary: defaultdict(<class 'int'>, {'a </w>': 1, 's a i l o r </w>': 1, 'w e n t </w>': 1, 't o </w>': 2, 's e a </w>': 6, 's e e </w>': 7, 'w h a t </w>': 1, 'h e </w>': 2, 'c o u l d </w>': 2, 'b u t </w>': 1, 'a l l </w>': 1, 't h a t </w>': 1, 'w a s </w>': 1, 't h e </w>': 2, 'b o t t o m </w>': 1, 'o f </w>': 1, 'd e e p </w>': 1, 'b l u e </w>': 1})
Size of vocabulary: 18
Most frequent pair: ('s', 'e')
Most frequent pair: ('e', '</w>')
Most frequent pair: ('a', '</w>')
Most frequent pair: ('se', 'e</w>')
Most frequent pair: ('se', 'a</w>')
Most frequent pair: ('t', '</w>')
Most frequent pair: ('h', 'e</w>')
Most frequent pair: ('t', 'o')
Most frequent pair: ('to', '</w>')
Most frequent pair: ('h', 'a')
Most fr

<span style="color:green;white-space:pre-wrap">It differs initially by one because there is one key that contains only white space, but the vocab does not have white space token (key) because when constructing pairs split is used and it will remove all white spaces within a key from the vocab. It seems to converge to 18, which is expected, because of how the vocab and tokens are constructed.</span>

TODO - Consider the input text:

"How much wood could a woodchuck chuck if a woodchuck could chuck wood"

How many tokens will there be initially and what will they be?
<span style="color:green;white-space:pre-wrap">Initially the tokens are letters including the white space token. So there are 14 different latters in the text if we also include white space as distinct character, while the vocab will just capture the distinct words with the only difference that they are padded with space and some kind of word tag </w>, but in this case since there are 8 distinct words, the size of vocab is therefore 8.</span>
How many tokens will there be if we run the tokenization routine for the maximum number of iterations (merges)?
<span style="color:green;white-space:pre-wrap">When it converges (24 steps) the number of tokens should be the same i.e. 8, it's just that each token will now be an actual word.</span>
Why does this algorithm work?
<span style="color:green;white-space:pre-wrap">The gist of it is that it merges the most common adjacent pair each time given the current vocab content e.g. all occurrences of 's e' become 'se', but if a token is just a whitespace it will not merge with another token/character/subword/word that ends or starts at whitespace because otherwise it would merge across words, which we want to avoid. The only criteria to merge is that there is a subword that is surrounded by white space, and we check every relevant subword in the text. So, eventually all subwords within a key (in the vocab) will be merged, because we gradually are removing whitespaces from them. As mentioned the error after convergence occurs because the pairs dictionary will be empty when all words have been merged, because there won't be any white spaces to split, therefore the loop that creates the pairs will never run. So when we try to take max frequency of pairs, we are essentially trying to access an empty dictionary which raises an error.</span>

When you've made your predictions, run the code and see if you are correct.

In [14]:
woody_text = "How much wood could a woodchuck chuck if a woodchuck could chuck wood"
tokens, vocab = tokenize(woody_text, num_merges=24)
print('Tokens: {}'.format(tokens))
print('Number of tokens: {}'.format(len(tokens)))
print('Vocabulary: {}'.format(vocab))
print('Size of vocabulary: {}'.format(len(vocab)))

Initial stats of size of vocab and number of tokens
Tokens: defaultdict(<class 'int'>, {'H': 1, 'o': 11, 'w': 5, '</w>': 13, 'm': 1, 'u': 7, 'c': 11, 'h': 5, 'd': 6, 'l': 2, 'a': 2, 'k': 4, 'i': 1, 'f': 1})
Number of tokens: 14
Vocabulary: defaultdict(<class 'int'>, {'H o w </w>': 1, 'm u c h </w>': 1, 'w o o d </w>': 2, 'c o u l d </w>': 2, 'a </w>': 2, 'w o o d c h u c k </w>': 2, 'c h u c k </w>': 2, 'i f </w>': 1})
Size of vocabulary: 8
Most frequent pair: ('u', 'c')
Most frequent pair: ('w', 'o')
Most frequent pair: ('wo', 'o')
Most frequent pair: ('woo', 'd')
Most frequent pair: ('c', 'h')
Most frequent pair: ('ch', 'uc')
Most frequent pair: ('chuc', 'k')
Most frequent pair: ('chuck', '</w>')
Most frequent pair: ('wood', '</w>')
Most frequent pair: ('c', 'o')
Most frequent pair: ('co', 'u')
Most frequent pair: ('cou', 'l')
Most frequent pair: ('coul', 'd')
Most frequent pair: ('could', '</w>')
Most frequent pair: ('a', '</w>')
Most frequent pair: ('wood', 'chuck</w>')
Most freque